<a href="https://colab.research.google.com/github/ldj7672/LLM-Tutorials/blob/main/examples/OpenAI_RAG_ChatAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OpenAI RAG 실습**

**OpenAI**의 **Chat API**를 사용하여, 간단한 **RAG** 기능을 구현해보는 실습 코드입니다.

## **1. 환경 세팅**
- OpenAI 라이브러리 설치
- API Key 입력
- 구글 드라이브 마운트

In [ ]:
!pip install -q openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 920.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.5 MB/s eta 

In [ ]:
from getpass import getpass
import os
from bs4 import BeautifulSoup
import chromadb
from openai import OpenAI
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [ ]:
# Colab에 API 키를 안전하게 입력받기
api_key = getpass("OpenAI API 키를 입력하세요: ")
os.environ['OPENAI_API_KEY'] = api_key

OpenAI API 키를 입력하세요: ··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **2. 함수 선언**
- `extract_text_from_html` : HTML 파일에서 텍스트를 추출
- `smart_chunk_splitter` : 텍스트 길이에 따라 분할하는 함수
- `create_chat_prompt` : 프롬프트 생성 함수
- `generate_response` : OpenAI API 응답 생성
- `chat_with_rag` : RAG 기반 응답 생성 함수

In [ ]:
# HTML 파일에서 텍스트 추출 함수
def extract_text_from_html(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # 모든 텍스트를 추출하며, 필요에 따라 특정 태그를 대상으로 수정할 수 있음
        text = ' '.join(soup.stripped_strings)
    return text

# 텍스트를 길이에 따라 자연스럽게 분할하는 함수
def smart_chunk_splitter(text, max_chunk_size=500):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '

    if current_chunk:  # 남아 있는 텍스트 추가
        chunks.append(current_chunk.strip())

    return chunks

# 프롬프트 생성 함수
def create_chat_prompt(system_prompt, user_query, context_documents):
    # context_documents가 중첩 리스트일 경우 평탄화
    if isinstance(context_documents[0], list):
        context_documents = [item for sublist in context_documents for item in sublist]

    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': f"""
            Context:
            {" ".join(context_documents)}

            Context를 참조해서 아래 질문에 답해주세요.

            Question:
            {user_query}

            Answer:
        """}
    ]

# OpenAI API를 통한 응답 생성 함수
def generate_response(system_prompt, user_query, context_documents):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=create_chat_prompt(system_prompt=system_prompt, user_query=user_query, context_documents=context_documents),
        max_tokens=1500  # 필요한 만큼 토큰 수 조정
    )
    return response.choices[0].message.content

# RAG 기반 응답 생성 함수
def chat_with_rag(system_prompt, user_query):
    query_result = corpus_collection.query(query_texts=[user_query], n_results=8)
    print(query_result['documents'])
    response = generate_response(system_prompt, user_query, query_result['documents'])
    return response

## **4. 모델 선언**
- 텍스트 임베딩 모델, GPT 모델 선언

In [ ]:
# models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-4o-mini"

# ChromaDB 임베딩 함수 초기화
embedding_function = OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name=EMBEDDING_MODEL
)

client = OpenAI()

## **5. RAG 참조 문서**
- html 파일 경로 지정
- html 파일의 텍스트를 읽고, 지정된 chunk size로 분할



In [ ]:
# HTML 파일 경로
html_path = "/content/drive/MyDrive/코드 예제/LLM/ref_docs/KBO_리그.html"

# 텍스트 추출 및 분할
text = extract_text_from_html(html_path)
chunks = smart_chunk_splitter(text, max_chunk_size=2000)

# 분할된 텍스트 출력
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
KBO 리그 - 나무위키 최근 변경 최근 토론 특수 기능 KBO 리그 최근 수정 시각: 2024-09-15 11:19:51 490 편집 토론 역사 분류 KBO 리그 은(는) 여기로 연결됩니다. 해당 리그의 운영 주체에 대한 내용은 한국야구위원회 문서 를 의 번 문단을 의 부분을 , 슈퍼 패미컴용으로 출시한 게임에 대한 내용은 한국프로야구(게임) 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 참고하십시오. 2024년 시즌에 대한 내용은 2024 신한 SOL Bank KBO 리그 문서 를 의 번 문단을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분을 의 부분을 , 에 대한 내용은 문서 를 의 번 문단을 의 번 문단을 의 부분

## **6. ChromaDB 세팅**
- ChromaDB 클라이언트 및 컬렉션 초기화
- 각 텍스트 청크를 컬렉션에 추가

In [ ]:
# ChromaDB 클라이언트 및 컬렉션 초기화
chroma_client = chromadb.Client()  # 메모리 기반 상태로 실행
corpus_collection = chroma_client.create_collection(
    name='KBO',
    embedding_function=embedding_function
)

In [ ]:
# 각 텍스트 청크를 컬렉션에 추가
for i, chunk in enumerate(chunks):
    corpus_collection.add(ids=[str(i)], documents=[chunk])

## **7. RAG 기반 대화 진행**
- system_prompt, user_query 입력

In [ ]:
# system prompt 설정
system_prompt = "KBO 관련 질문에 사실 기반으로 답변하는 어시스턴트입니다."

In [ ]:
# 예제 질문에 대한 RAG 응답 생성
user_query = "KBO 구단별 매출에 대해 간략하게 요약해줘"
response = chat_with_rag(system_prompt, user_query)
response

[['2023년에는 시즌 전에 개최된 2023 월드 베이스볼 클래식 에서 이강철호 의 3대회 연속 1라운드 탈락, 특히 일본전 참패의 여파로 인해 리그 흥행에 심각한 후폭풍을 초래할지도 모른다는 우려도 나왔으나 오히려 개막전이 전 경기 매진되는 등 흥행에 순풍이 불고 있어서 이전에도 그랬듯이 위기론은 시기상조라는 의견이 중론이다. 한국에서 해당 대회 최고 시청률은 한일전에서 기록한 11.7%로, 대회 자체에 사람들이 별 관심을 두지 않았다. 2022년 한국시리즈 6차전 시청률 8.0%와도 큰 차이가 나지 않는다. 오히려 2020 올림픽의 야구 시청률이 훨씬 높았다. 당시에는 대부분의 경기가 시청률 20%를 넘겼고, 한일전은 시청률 28%를 기록했다. 월드 베이스볼 클래식 이라는 대회는 사실 인지도가 올림픽이나 월드컵에 비해 낮고, 한국이 3연속 조별탈락하면서 국민들의 관심에서 멀어졌다고도 볼 수 있다. 2023 시즌 중반기를 넘어 후반기로 향하고 있는 지금, 8월 3일 기준으로 관중 수는 전년대비 34%나 증가했고, 코로나 이전인 2019년과 대비해서도 3.4% 증가하였다. 또한 TV 시청률도 0.864%로, 작년 동일 경기수 대비 6% 상승하면서 위에 서술된 올해의 KBO에서 일어난 사건사고들이 흥행에는 사실상 영향을 전혀 주지 않고 있다는 것을 알 수 있다. 애초에 사건사고나 국제 대회 성적에 인기가 휘둘리는 리그가 비정상적이고, 이런 모습이 자연스러운 것이다. [39] 2023 KBO 리그 500 만 관중 달성 임박 전년 대비 TV중계 시청률 상승 2023년에 야구계에 여러가지 악재가 있었지만 시즌 막바지인 10월 15일에는 800만 관중 돌파에 성공했다. # 2018년 이후 5년만의 800만 관중 돌파이며, KBO 리그 창설 이후 통산 4번째 800만 돌파이다. 최종 관중수는 810만 명으로 KBO 관중수 역대 3위를 기록했다. 2023년 조사한 국내 인기 스포츠 팀 순위. Top10중 7팀이 KBO 리그의 팀이다. 2024년에는 지난해보다 압도적인 수

'KBO 구단들 대부분은 모기업의 지원을 받으며 운영되지만, 매출은 몇 백억 원에 이르지만 순이익은 억대 이하이거나 적자인 경우가 많습니다. 이는 매출액이 모기업 지원금을 포함한 것이라, 실제로 구단들은 운영에 있어 상당한 적자를 감내해야 합니다. 특히, 키움 히어로즈는 모기업이 없는 특성 덕분에 독특한 구조를 가지고 있으며, 자기 수익 중심으로 운영되고 있습니다. 기업들은 야구단 운영을 통해 얻는 보이지 않는 경제적 가치를 중시하여 계속해서 구단 운영을 이어가고 있으며, 이는 구단 경영의 중요한 동기입니다.'

In [ ]:
# 예제 질문에 대한 RAG 응답 생성
user_query = "한국 시리즈에서 우승하지 못한 팀을 알려줘"
response = chat_with_rag(system_prompt, user_query)
response

[['우승 및 준우승 횟수 [편집] 팀 우승 준우승 우승 연도 준우승 연도 통산 전적 비고 KIA 타이거즈 11 [192] 0 1983, 1986, 1987, 1988, 1989, 1991, 1993, 1996, 1997, 2009, 2017 - 59전 44승 2무 13패 승률: 0.772 연승: 10연승 [193] [194] 삼성 라이온즈 8 10 1985, 2002, 2005, 2006, 2011, 2012, 2013, 2014 1982, 1984, 1986, 1987, 1990, 1993, 2001 , 2004, 2010, 2015 97전 40승 6무 51패 승률: 0.440 연승: 5연승 [195] [196] 두산 베어스 6 9 1982, 1995 , 2001, 2015, 2016, 2019 2000, 2005, 2007, 2008, 2013, 2017, 2018 , 2020, 2021 82전 38승 1무 43패 승률: 0.469 연승: 9연승 [197] [198] SSG 랜더스 5 4 2007, 2008, 2010, 2018, 2022 2003, 2009, 2011, 2012 52전 29승 23패 승률: 0.558 연승: 4연승 [199] [200] 현대 유니콘스 4 2 1998 , 2000, 2003, 2004 1994, 1996 39전 18승 3무 18패 승률: 0.500 연승: 4연승 [201] [202] LG 트윈스 3 4 1990, 1994, 2023 1983, 1997, 1998, 2002 35전 18승 17패 승률: 0.514 연승: 8연승 [203] [204] 롯데 자이언츠 2 3 1984, 1992 1985, 1995, 1999 24전 12승 12패 승률: 0.500 연승: 4연승 [205] [206] 한화 이글스 1 5 1999 1988, 1989 , 1991, 1992 , 2006 31전 10승 21패 승률: 0.323 연승: 2연승 [207] [208] NC 다이노스 1 1 2020 2016 10전 4승 6패 승률: 0.400 연

'한국 시리즈에서 우승하지 못한 팀은 키움 히어로즈가 유일합니다.'